In [1]:
import cv2
import numpy as np
from sklearn.svm import SVC
import os
import sys
import time
from scipy.signal import convolve


# EXTRACTOR 1

| Dataset	   |Precisión  | Tiempo de ejecución|
|-------------- |-----------|------------|
| "cat_dog_100" | 95%     | 16.30 segundos      |
| "cat_dog_500"     | 92,5% | 4 minutos 26.28 segundos    |

In [2]:
def binarioPro(ul,u,ur,l,pix,r,dl,d,dr):
    res = np.zeros(pix.shape)
    vecinos = np.array([ul,u,ur,r,dr,d,dl,l])
    for i in range(len(vecinos)):
        res = res + np.power(2,i)*(vecinos[i] >=pix)
    return res

In [3]:
def feature_extraction(imagen, N):
    imagen_amp = cv2.copyMakeBorder(imagen, 1, 1, 1, 1, cv2.BORDER_REPLICATE)
    newIm = np.zeros(imagen.shape)    

    newIm = binarioPro(imagen_amp[0:-2,0:-2],imagen_amp[0:-2,1:-1],imagen_amp[0:-2,2:],imagen_amp[1:-1,0:-2],imagen_amp[1:-1,1:-1],imagen_amp[1:-1,2:],imagen_amp[2:,0:-2],imagen_amp[2:,1:-1],imagen_amp[2:,2:])
    
    height, width = imagen.shape[:2]

    # Calculamos el número de bloques en filas y columnas
    
    num_blocks_rows = height // N
    num_blocks_cols = width // N

    # Dividimos la imagen en bloques de 16 filas
    blocks = np.vsplit(imagen, num_blocks_rows)

    # Creamos una lista para guardar todas las celdas   
    histogramas = np.zeros((num_blocks_rows*num_blocks_cols,256))
    
    # Iteramos sobre los bloques
    for i, block in enumerate(blocks):
        # Dividimos cada bloque en bloques d
        # e 16 columnas
        cells = np.hsplit(block, num_blocks_cols)

        # Iteramos sobre las celdas
        for j, cell in enumerate(cells):
            # Añadimos cada celda a la lista    
            histogramas[j*i] = cv2.calcHist([cell], [0], None, [256], [0,256]).flatten()

        
    return histogramas.flatten()

In [8]:
datasets = ["cat_dog_100" ,"cat_dog_500"  ]

N = 16

HEIGHT =  384
WIDTH = 528

num_blocks_rows = HEIGHT // N
num_blocks_cols = WIDTH // N

for dataset in datasets:
    print("dataset: ", dataset)
    # Iniciamos el contador de tiempo
    startTimer = time.perf_counter()

    # Definimos el directorio de entrenamiento y prueba
    train_dir = dataset+"/"+"train"
    test_dir = dataset+"/"+"test"

    # Inicializamos los arrays de datos de entrenamiento y prueba
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    print("Procesamos las imágenes de entrenamiento")
    # Procesamos las imágenes de entrenamiento
    for label in os.listdir(train_dir):
        label_dir = os.path.join(train_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path , 0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)            
            
            
            features = feature_extraction(resized_image, N);       

            X_train.append(features.flatten())
            y_train.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtrain ... (" + str(int(progress)) + " %)")
        print("")

    print("Procesamos las imágenes de prueba")
    # Procesamos las imágenes de prueba
    for label in os.listdir(test_dir):
        label_dir = os.path.join(test_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path,0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
                
            features = feature_extraction(resized_image, N);   

            
            X_test.append(features.flatten())
            y_test.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtest ... (" + str(int(progress)) + " %)")
        print("")
    print("")
    
    # Convertimos los datos de entrenamiento y prueba a arrays de NumPy
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)


    print("SHAPE Xtrin",X_train.shape)
    print("SHAPE y_train",y_train.shape)
    print("SHAPE X_test",X_test.shape)
    print("SHAPE y_test",y_test.shape)
    
    
    # Creamos un clasificador SVM
    clf = SVC(gamma='auto')

    # Entrenamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, y_train)

    # Realizamos predicciones con el clasificador entrenado
    y_pred = clf.predict(X_test)

    # Calculamos el porcentaje de acierto
    accuracy = np.mean(y_pred == y_test)
    print("Precisión: {:.2f}%".format(accuracy * 100))

    # Detenemos el contador de tiempo
    endTimer = time.perf_counter()

    # Calculamos el tiempo transcurrido
    elapsed = endTimer - startTimer
    # Obtenemos el tiempo transcurrido en minutos y segundos
    minutes, seconds = divmod(elapsed, 60)

    # Mostramos el tiempo transcurrido en minutos y segundos
    print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.2f} segundos")

    
    


dataset:  cat_dog_100
Procesamos las imágenes de entrenamiento
Procesando label:  cat
 Procesando Xtrain ... (100 %)
Procesando label:  dog
 Procesando Xtrain ... (100 %)
Procesamos las imágenes de prueba
Procesando label:  cat
 Procesando Xtest ... (100 %)
Procesando label:  dog
 Procesando Xtest ... (100 %)

SHAPE Xtrin (200, 202752)
SHAPE y_train (200,)
SHAPE X_test (40, 202752)
SHAPE y_test (40,)
Precisión: 95.00%
Tiempo transcurrido: 0 minutos 19.58 segundos
dataset:  cat_dog_500
Procesamos las imágenes de entrenamiento
Procesando label:  cat
 Procesando Xtrain ... (100 %)
Procesando label:  dog
 Procesando Xtrain ... (100 %)
Procesamos las imágenes de prueba
Procesando label:  cat
 Procesando Xtest ... (100 %)
Procesando label:  dog
 Procesando Xtest ... (100 %)

SHAPE Xtrin (1000, 202752)
SHAPE y_train (1000,)
SHAPE X_test (200, 202752)
SHAPE y_test (200,)
Precisión: 92.50%
Tiempo transcurrido: 4 minutos 53.17 segundos


# EXTRACTOR 2

| Dataset	   |Precisión  | Tiempo de ejecución|
|-------------- |-----------|------------|
| "cat_dog_100" | 92,5%     | 16.46 segundos       |
| "cat_dog_500"     | 90% | 1 minutos 27.68 segundos      |

In [4]:
def crearCeldas(image, N):
    height, width = image.shape[:2]

  # Calculamos el número de bloques en filas y columnas
    num_blocks_rows = height // N
    num_blocks_cols = width // N

    # Dividimos la imagen en bloques de 16 filas
    blocks = np.vsplit(image, num_blocks_rows)

    # Creamos una lista para guardar todas las celdas
    celdas = []

    # Iteramos sobre los bloques
    for i, block in enumerate(blocks):
        # Dividimos cada bloque en bloques d
        # e 16 columnas
        cells = np.hsplit(block, num_blocks_cols)

        # Iteramos sobre las celdas
        for j, cell in enumerate(cells):
            # Añadimos cada celda a la lista
            celdas.append(cell)
            

    celdas = np.array(celdas)
    return celdas

In [5]:
def feature_extraction_2(image, N, R):
    
    maskX = np.array([[0, 0, 0], [-1, 0, 1], [0, 0, 0]])
    maskY = np.array([[0, -1, 0], [0, 0, 0], [0, 1, 0]])   



    dx = convolve(image, maskX, mode='same')
    dy = convolve(image, maskY, mode='same')

    magnitudes = np.uint8(np.sqrt(np.square(dx) + np.square(dy)))
    
    

    PhiRad = np.arctan2(dy , dx)
    phi = np.uint8(np.rad2deg(PhiRad))

    orientaciones = np.abs(phi)//20
    celdasMag = crearCeldas(magnitudes, N)

    celdasMask = np.zeros((9,celdasMag.shape[0] ,celdasMag.shape[1] ,celdasMag.shape[2]))
    for i in range(celdasMask.shape[0]):         
        celdasMask[i] = crearCeldas(orientaciones == i ,N)   


    histograms = np.zeros((len(celdasMag), 9))
 
 
    histograms = np.sum(celdasMask * celdasMag[:] , axis = (2,3)) 
    
    valor = np.sqrt(np.sum(histograms**2,axis=0)) 

    histograms = np.where(valor != 0, histograms / valor, histograms)

    vector = histograms.reshape(1,-1)
    
    

    return vector.flatten()

In [6]:
datasets = ["cat_dog_100" , "cat_dog_500"]

N = 16
R = 3

HEIGHT = 384
WIDTH = 528

for dataset in datasets:
    print("dataset: ", dataset)
    # Iniciamos el contador de tiempo
    startTimer = time.perf_counter()

    # Definimos el directorio de entrenamiento y prueba
    train_dir = dataset+"/"+"train"
    test_dir = dataset+"/"+"test"

    # Inicializamos los arrays de datos de entrenamiento y prueba
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    print("Procesamos las imágenes de entrenamiento")
    # Procesamos las imágenes de entrenamiento
    for label in os.listdir(train_dir):
        label_dir = os.path.join(train_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(
                image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
                
            features = feature_extraction_2(resized_image, N, R)
            X_train.append(features)
            y_train.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtrain ... (" + str(int(progress)) + " %)")
        print("")

    print("Procesamos las imágenes de prueba")
    # Procesamos las imágenes de prueba
    for label in os.listdir(test_dir):
        label_dir = os.path.join(test_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(
                image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
            features = feature_extraction_2(resized_image, N, R)
            X_test.append(features)
            y_test.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtest ... (" + str(int(progress)) + " %)")
        print("")
        
    print("")
    # Convertimos los datos de entrenamiento y prueba a arrays de NumPy
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)    


    # Creamos un clasificador SVM
    clf = SVC(C=5,gamma=0.01)

    # Entrenamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, y_train)

    # Realizamos predicciones con el clasificador entrenado
    y_pred = clf.predict(X_test)

    # Calculamos el porcentaje de acierto
    accuracy = np.mean(y_pred == y_test)
    print("Precisión: {:.2f}%".format(accuracy * 100))

    # Detenemos el contador de tiempo
    endTimer = time.perf_counter()

    # Calculamos el tiempo transcurrido
    elapsed = endTimer - startTimer
    # Obtenemos el tiempo transcurrido en minutos y segundos
    minutes, seconds = divmod(elapsed, 60)

    # Mostramos el tiempo transcurrido en minutos y segundos
    print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.2f} segundos")


    


dataset:  cat_dog_100
Procesamos las imágenes de entrenamiento
Procesando label:  cat


IndexError: arrays used as indices must be of integer (or boolean) type

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

C_values = [0.1, 1.0, 5.0, 7.5, 10.0]
gamma_values = [0.001, 0.01, 0.1, 1, 10]

# Genera el diccionario param_grid con los valores indicados
param_grid = {'C': C_values, 'gamma': gamma_values}

# Genera el objeto ShuffleSplit para generar un único particionado fijo, con 20% de datos de validación y
# random_state = 0
rs = ShuffleSplit(n_splits=1, test_size=.20, random_state=0)

# Haz la llamada a la función GridSearchCV con un clasificador SVC y el diccionario de parámetros definido
# Para acelerar algo el entrenamiento, emplea también el parámetro n_jobs=-1, que hará que se aproveche mejor el
# procesador de tu equipo.
clasificadores = GridSearchCV(SVC(), param_grid, n_jobs=-1, cv=rs)

# Entrena el modelo interno de GridSearchCV, y después obten los mejores valores de C y gamma en C_best y gamma_best
clasificadores.fit(X_train, y_train)

C_best = clasificadores.best_params_["C"]
gamma_best = clasificadores.best_params_["gamma"]

print("Mejores parámetros C: {}, gamma: {}".format(C_best, gamma_best))

# Crea y entrena un modelo SVC con C_best y gamma_best sobre los datos de train originales (X e y)
clf = SVC(C=C_best, gamma=gamma_best)
clf.fit(X_train, y_train)

# Realizamos predicciones con el clasificador entrenado
y_pred = clf.predict(X_test)

# Calculamos el porcentaje de acierto
accuracy = np.mean(y_pred == y_test)
print("Precisión: {:.2f}%".format(accuracy * 100))

# Detenemos el contador de tiempo
endTimer = time.perf_counter()

# Calculamos el tiempo transcurrido
elapsed = endTimer - startTimer
# Obtenemos el tiempo transcurrido en minutos y segundos
minutes, seconds = divmod(elapsed, 60)

# Mostramos el tiempo transcurrido en minutos y segundos
print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.2f} segundos")


Mejores parámetros C: 5.0
Precisión: 92.50%
Tiempo transcurrido: 12 minutos 29.80 segundos
